In [1]:
import spacy
from spacy import displacy
from spacy.matcher import Matcher
from spacy.tokens import Span
nlp = spacy.load('en_core_web_sm')
from string import punctuation

In [2]:

# doc = nlp("Bake for 55 to 60 minutes, or until filling is set")
doc = nlp('Bake for 55 to 60 minutes, then rest for 45 to 55 minutes')
for token in doc:
    print(token, token.text, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop, token.children, token.head)
#     print(token.head, token.text)
#     displacy.render(doc, style="dep")


Bake Bake VERB VB ROOT Xxxx True False <generator object at 0x000002D225792AF0> Bake
for for ADP IN prep xxx True True <generator object at 0x000002D225792AF0> Bake
55 55 NUM CD quantmod dd False False <generator object at 0x000002D20902DA60> 60
to to PART TO quantmod xx True True <generator object at 0x000002D20902DA60> 60
60 60 NUM CD nummod dd False False <generator object at 0x000002D20902DA60> minutes
minutes minutes NOUN NNS pobj xxxx True False <generator object at 0x000002D20902DA60> for
, , PUNCT , punct , False False <generator object at 0x000002D20902DA60> Bake
then then ADV RB advmod xxxx True True <generator object at 0x000002D20902DA60> Bake
rest rest VERB VB dep xxxx True False <generator object at 0x000002D20902DA60> Bake
for for ADP IN prep xxx True True <generator object at 0x000002D20902DA60> rest
45 45 NUM CD quantmod dd False False <generator object at 0x000002D20902DA60> 55
to to PART TO quantmod xx True True <generator object at 0x000002D20902DA60> 55
55 55 NUM C

In [3]:
def findMethod(step):
    newstep = step

    doc = nlp(newstep)
    methods = {}
    timescale = {}
    times = {}
    connector = {}
    for i, entity in enumerate(doc):
        if entity.pos_ == "VERB":  #and str(entity.head) == str(entity.text)
            methods[entity] = {'times':[], 'scale':None}
        if entity.dep_ == "pobj":
            timescale[entity] = entity.head
        if entity.pos_ == "NUM" and (entity.dep_ == 'quantmod' or entity.dep_ == 'nummod'):
            times[entity] = entity.head
        for m in methods:
            if entity.head == m and str(entity.text) != str(entity.head):
                connector[entity] = m
    
    for scale in timescale:
        while timescale[scale] not in methods:
            if not connector[timescale[scale]]:
                print("Error in dependency parsing")
                break
            timescale[scale] = connector[timescale[scale]]
            
    for time in times:
        if times[time] in times:
            times[time] = times[times[time]]
    
    for s in timescale:
        for t in times:
            if times[t] == s:
                methods[timescale[s]]['times'].append(float(str(t)))
        methods[timescale[s]]['scale'] = str(s)
        
    for m in methods:
        if methods[m]['scale'] == 'seconds':
            methods[m]['times'] = [t/60 for t in methods[m]['times']]
            methods[m]['scale'] = 'minutes'
        elif methods[m]['scale'] == 'hours':
            methods[m]['times'] = [t*60 for t in methods[m]['times']]
            methods[m]['scale'] = 'minutes'
    
    ## get indices of times, pobj (minutes, hours, days, etc), and method, and use distance to see which ones belong to which one.

    return methods

In [4]:
m = findMethod('Bake for 55 to 60 minutes, then cook for 1.5 hours')
print(m)

{Bake: {'times': [55.0, 60.0], 'scale': 'minutes'}, cook: {'times': [90.0], 'scale': 'minutes'}}


In [7]:
print(m)
print(t)

{'filling': 2, 'minutes': 5}
{'55': 8, 'Bake': 10}
